# 단어 벡터에 대한 연산

이번 주 첫 번째 과제에 오신 것을 환영합니다!

단어 임베딩은 훈련하는 데 계산적으로 매우 비싸기 때문에 대부분의 ML 실무자는 사전 훈련된 임베딩 세트를 로드합니다.

**이 과제 후에 다음을 수행 할 수 있습니다. **

- 사전 훈 된 단어 벡터를 로드하고 코사인 유사성을 사용하여 유사성을 측정합니다.
- 단어 임베딩을 사용하여 Woman 에 대한 Man 같이 ____ 에 대한 King 과 같이 단어 비유 문제를 해결합니다.
- 성별 편견을 줄이기 위해 단어 임베딩 수정

시작합시다! 다음 셀을 실행하여 필요한 패키지를 로드하십시오.

In [1]:
import numpy as np
from w2v_utils import *

다음에는, 단어 벡터를 로드합니다. 이 과제를 위해 50 차원 GloVe 벡터를 사용하여 단어를 표현합니다. 다음 셀을 실행하여`word_to_vec_map`을 로드합니다.

In [2]:
words, word_to_vec_map = read_glove_vecs('./data/glove.6B.50d.txt')

다음을 로드했습니다 :

- `words`: 어휘의 단어 집합입니다.
- `word_to_vec_map` : 단어를 GloVe 벡터 표현에 매핑하는 사전.

당신은 원-핫 벡터가 어떤 단어가 유사한 지 알아내는 데 좋은 역할을 하지 않는다는 것을 보았습니다. GloVe 벡터는 개별 단어의 의미에 대해 훨씬 더 유용한 정보를 제공합니다. 이제 GloVe 벡터를 사용하여 두 단어가 얼마나 유사한 지 결정하는 방법을 살펴 보겠습니다.

# 1. 코사인 유사도

두 단어가 얼마나 유사한 지 측정하려면 두 단어에 대한 두 임베딩 벡터 간의 유사도를 측정하는 방법이 필요합니다. 두 벡터 $u$ 및 $v$가 주어지면 코사인 유사성은 다음과 같이 정의됩니다.

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

여기서 $uv$는 두 벡터의 내적 (또는 내적)이고  $||u||_2$ 는 벡터 $u$ 의 norm (또는 길이)이며 $\theta$는 $ u$ 와 $ v $ 사이의 각도입니다. 이 유사성은 $ u $와 $ v $ 사이의 각도에 따라 다릅니다. $ u $와 $ v $가 매우 유사하면 코사인 유사성은 1에 가깝습니다. 유사하지 않은 경우 코사인 유사도는 더 작은 값을 사용합니다.

<img src="images/cosine_sim.png" style="width:800px;height:250px;">
<caption><center> Figure 1: 두 벡터 사이 각도의 코사인은 두 벡터가 얼마나 유사한 지 측정합니다.</center></caption>

**(1) 연습 과제** : 단어 벡터 간의 유사성을 평가하기 위해 `cosine_similarity()` 함수를 구현합니다.

**알림** : $ u $의 표준은 $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$ 로 정의됩니다.

In [3]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    
    # Compute the dot product between u and v (≈1 line)
    dot = np.dot(u,v)
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u*u))
    
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v*v))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cosine_similarity = dot/(norm_u*norm_v)
    
    return cosine_similarity

In [4]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.2743924626137942
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174201


**Expected Output**:

<table>
    <tr>
        <td>
            cosine_similarity(father, mother) =
        </td>
        <td>
         0.890903844289
        </td>
    </tr>
        <tr>
        <td>
            cosine_similarity(ball, crocodile) =
        </td>
        <td>
         0.274392462614
        </td>
    </tr>
        <tr>
        <td>
            cosine_similarity(france - paris, rome - italy) =
        </td>
        <td>
         -0.675147930817
        </td>
    </tr>
</table>

올바른 예상 출력을 얻은 후 자유롭게 입력을 수정하고 다른 단어 쌍 간의 코사인 유사성을 측정하십시오! 다른 입력의 코사인 유사성을 둘러 보면 단어 벡터가 어떻게 작동하는지 더 잘 이해할 수 있습니다.

## 2. 단어 비유 작업

단어 비유 과제에서 우리는 <font color='brown'>"*a* is to *b* as *c* is to **____**"</font> 문장을 완성합니다. 예를 들면 <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font> 입니다. 구체적으로, 우리는 관련 단어 벡터 $e_a, e_b, e_c, e_d$ 가 $e_b - e_a \approx e_d - e_c$ 와 같은 방식으로 관련되도록 단어 *d*를 찾으려고 합니다. 코사인 유사성을 사용하여 $ e_b-e_a $ 과 $ e_d-e_c $ 간의 유사성을 측정합니다.

**(2) 연습 과제** : 단어 비유를 수행하려면 아래 코드를 완성하세요!

In [5]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lower case
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output

    # loop over the whole word vector set
    for w in words:        
        # to avoid best_word being one of the input words, pass on them.
        if w in [word_a, word_b, word_c] :
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity(e_b-e_a, word_to_vec_map[w]-e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        ### END CODE HERE ###
        
    return best_word

아래 셀을 실행하여 코드를 테스트하세요. 1 ~ 2 분 정도 걸릴 수 있습니다.

In [7]:
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large'), ('small', 'smaller', 'big')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad,word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> larger
small -> smaller :: big -> competitors


**Expected Output**:

<table>
    <tr>
        <td>
            italy -> italian ::
        </td>
        <td>
         spain -> spanish
        </td>
    </tr>
        <tr>
        <td>
            india -> delhi ::
        </td>
        <td>
         japan -> tokyo
        </td>
    </tr>
        <tr>
        <td>
            man -> woman  ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            small -> smaller  ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

올바른 예상 출력을 얻으면 위의 입력 셀을 자유롭게 수정하여 자신의 유추를 테스트하십시오. 작동하는 다른 비유 쌍을 찾으십시오. 그러나 알고리즘이 정답을 제공하지 않는 부분도 찾으십시오. 예를 들어 small-> smaller as big-> ?.

### 축하합니다!

이 과제를 마치셨습니다. 기억해야 할 주요 사항은 다음과 같습니다.

- 코사인 유사성은 단어 벡터 쌍 간의 유사성을 비교하는 좋은 방법입니다. (L2 거리도 작동하지만)
- NLP 애플리케이션의 경우 인터넷에서 사전 학습 된 단어 벡터 세트를 사용하는 것이 시작하기에 좋은 방법 인 경우가 많습니다.

등급이 매겨진 부분을 마쳤더라도이 노트북의 나머지 부분도 살펴 보는 것이 좋습니다.

이 노트의 등급이 매겨진 부분을 완료 한 것을 축하합니다!

## 3. 단어 벡터 비편향화(Debiasing) 

다음 연습에서는 단어 임베딩에 반영 될 수있는 성별 편견을 조사하고 편향을 줄이기 위한 알고리즘을 탐색합니다. 편향성에 대한 주제에 대해 배우는 것 외에도 이 연습은 단어 벡터가 하는 일에 대한 직관을 연마하는데도 도움이 됩니다. 이 섹션에는 약간의 선형 대수가 포함되어 있지만 선형 대수에 대한 전문 지식이 없어도 완료 할 수 있으므로 한 번 시도해 보는 것이 좋습니다. 

먼저 GloVe 단어 임베딩이 성별과 어떤 관련이 있는지 살펴 보겠습니다. 먼저 $g = e_{woman}-e_{man}$ 벡터를 계산합니다. 여기서 $e_{woman}$ 는 단어 *woman* 에 해당하는 단어 벡터를 나타내고 $e_{man}$ 는 *man* 이라는 단어에 해당하는 벡터입니다. 결과 벡터 $ g $는 "gender"의 개념을 대략적으로 인코딩합니다. ($g_1 = e_{mother}-e_{father}$, $g_2 = e_{girl}-e_{boy}$ 등을 계산하고 평균을 계산하면 더 정확한 표현을 얻을 수 있습니다. 하지만 $e_{woman}-e_{man}$ 만 사용해도 당분간 충분한 결과를 제공합니다.)

In [8]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


이제 $ g $를 사용하는 다른 단어의 코사인 유사성을 고려할 것입니다. 유사성의 양의 값과 음의 코사인 유사성이 무엇을 의미하는지 고려하십시오.

In [9]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.315597935396073
sophie 0.31868789859418784
ronaldo -0.31244796850329437
priya 0.17632041839009402
rahul -0.16915471039231716
danielle 0.24393299216283895
reza -0.07930429672199553
katy 0.2831068659572615
yasmin 0.23313857767928758


보시다시피 여성 이름은 구성된 벡터 $g$ 와 양의 코사인 유사성을 갖는 경향이 있는 반면, 남성 이름은 음의 코사인 유사성을 갖는 경향이 있습니다. 이것은 놀라운 일이 아니며 결과는 받아 들일 수 있습니다.

그러나 다른 단어로 시도해 봅시다.

In [10]:
print('Other words and their similarities:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:
lipstick 0.2769191625638267
guns -0.1888485567898898
science -0.06082906540929701
arts 0.008189312385880337
literature 0.06472504433459932
warrior -0.20920164641125288
doctor 0.11895289410935041
tree -0.07089399175478091
receptionist 0.33077941750593737
technology -0.13193732447554302
fashion 0.03563894625772699
teacher 0.17920923431825664
engineer -0.0803928049452407
pilot 0.0010764498991916937
computer -0.10330358873850498
singer 0.1850051813649629


놀라운 점이 있습니까? 이러한 결과가 건강하지 않은 특정 성별 고정 관념을 반영하는 방식은 놀랍습니다. 예를 들어, "컴퓨터"는 "남자"에 더 가깝고 "문학"은 "여자"에 더 가깝습니다. 아야!

[Boliukbasi et al., 2016](https://arxiv.org/abs/1607.06520)로부터의 알고리즘을 사용하여 이러한 벡터의 편향을 줄이는 방법을 아래에서 살펴 보겠습니다. "배우"/"여배우"또는 "할머니"/"할아버지"와 같은 일부 단어 쌍은 성별에 따라 유지되어야하며, "수용자" 또는 "기술"과 같은 다른 단어는 중립화되어야 합니다. 즉, 성별과 관련이 없습니다. 비편향화할 때 이 두 가지 유형의 단어를 다르게 취급해야 합니다.

### 3.1. 성별이 아닌 단어에 대한 편향 중화(neutralize)

아래 그림은 중화가 수행하는 작업을 시각화하는 데 도움이 됩니다. 50 차원 단어 임베딩을 사용하는 경우 50 차원 공간은 바이어스 방향 $ g$ 와 나머지 49 차원의 두 부분으로 나눌 수 있습니다. $g_{\perp}$ 라고 합니다. 선형 대수학에서 우리는 49 차원 $g_{\perp}$ 가 $ g $에 수직 (또는 "othogonal")이라고 말하며, 이는 $ g $에 90도에 있음을 의미합니다. 중립화 단계는 $ e_{receptionist} $와 같은 벡터를 취하고 $ g $ 방향의 구성 요소를 0으로 만들어 $ e_{receptionist}^{debiased}$를 제공합니다.

$ g_{\ perp}$ 는 49 차원이지만 화면에 그릴 수 있는 것의 한계를 고려할 때 아래의 1 차원 축을 사용하여 설명합니다.

<img src="images/neutral.png" style="width:800px;height:300px;">
<caption><center> 그림 2 : 중립화 작업을 적용하기 전후에 표현 된 "receptionist"에 대한 단어 벡터. </center></caption>

**(3) 연습 과제** : `neutralize()`를 구현하여 'receptionist' 또는 'scientist' 와 같은 단어의 편견을 제거합니다. $ e$ 를 포함하는 입력이 주어지면 다음 공식을 사용하여 $e^{debiased}$ 를 계산할 수 있습니다.

$$e^{bias\_component} = \frac{e \cdot g}{||g||_2^2} * g\tag{2}$$
$$e^{debiased} = e - e^{bias\_component}\tag{3}$$

선형 대수학 전문가라면 $e^{bias\_component}$ 를 $ g $ 방향에 대한 $ e $ 투영으로 인식 할 수 있습니다. 선형 대수 전문가가 아니더라도 걱정하지 마십시오.

<!--
** 알림 ** : $ u $ 벡터는 벡터 축 $ v_B $에 대한 투영과 $ v $에 직교하는 축에 대한 투영의 두 부분으로 나눌 수 있습니다.
$$u = u_B + u_{\perp}$$
여기서 :  $u_B = $ 과 $ u_{\perp} = u - u_B $
!-->

In [11]:
def neutralize(word, g, word_to_vec_map):
    """
    Removes the bias of "word" by projecting it on the space orthogonal to the bias axis. 
    This function ensures that gender neutral words are zero in the gender subspace.
    
    Arguments:
        word -- string indicating the word to debias
        g -- numpy-array of shape (50,), corresponding to the bias axis (such as gender)
        word_to_vec_map -- dictionary mapping words to their corresponding vectors.
    
    Returns:
        e_debiased -- neutralized word vector representation of the input "word"
    """
    
    # Select word vector representation of "word". Use word_to_vec_map. (≈ 1 line)
    e = word_to_vec_map[word]
    
    # Compute e_biascomponent using the formula give above. (≈ 1 line)
    e_biascomponent = (np.dot(e,g)/np.linalg.norm(g)**2)*g
 
    # Neutralize e by substracting e_biascomponent from it 
    # e_debiased should be equal to its orthogonal projection. (≈ 1 line)
    e_debiased = e-e_biascomponent
    
    return e_debiased

In [12]:
e = "receptionist"
print("cosine similarity between " + e + " and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + " and g, after neutralizing: ", cosine_similarity(e_debiased, g))

cosine similarity between receptionist and g, before neutralizing:  0.33077941750593737
cosine similarity between receptionist and g, after neutralizing:  -2.099120994400013e-17


**Expected Output**: 두 번째 결과는 본질적으로 0 이며, 숫자 반올림($10^{-17}$ 정도) 까지입니다.


<table>
    <tr>
        <td>
            cosine similarity between receptionist and g, before neutralizing: :
        </td>
        <td>
         0.330779417506
        </td>
    </tr>
        <tr>
        <td>
            cosine similarity between receptionist and g, after neutralizing: :
        </td>
        <td>
         -3.26732746085e-17
    </tr>
</table>

### 3.2. 성별 단어에 대한 균등화 알고리즘

다음으로 "actress"와 "actor"와 같은 단어 쌍에도 편향성 제거를 적용하는 방법을 살펴 보겠습니다. 균등화는 성별 속성을 통해서만 다를 수 있는 단어 쌍에 적용됩니다. 구체적인 예로, "actress"가 "actor"보다 "babysit"에 더 가깝다고 가정합니다. "babysit"에 중화를 적용하면 베이비 시팅과 관련된 성별 입체형(gender-stereotype)을 줄일 수 있습니다. 그러나 이것이 "actor"와 "actress"가 "babysit"와 같은 거리에 있다는 것을 보장하지는 않습니다. 균등화(equalization) 알고리즘이 이를 처리합니다.

균등화의 핵심 아이디어는 특정 단어 쌍이 49 차원 $g_\perp$와 같은 거리에 있는지 확인하는 것입니다. 균등화 단계는 또한 두 균등화 단계가 이제 $e_{receptionist}^{debiased}$ 또는 중립화 된 다른 작업과 동일한 거리에 있는지 확인합니다. 그림에서 이퀄라이제이션이 작동하는 방식은 다음과 같습니다.

<img src="images/equalize10.png" style="width:800px;height:400px;">

이를 수행하기위한 선형 대수의 유도는 조금 더 복잡합니다. (자세한 내용은 Bolukbasi et al., 2016을 참조하십시오.) 그러나 주요 방정식은 다음과 같습니다.: 

$$ \mu = \frac{e_{w1} + e_{w2}}{2}\tag{4}$$ 

$$ \mu_{B} = \frac {\mu \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{5}$$ 

$$\mu_{\perp} = \mu - \mu_{B} \tag{6}$$

$$ e_{w1B} = \frac {e_{w1} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{7}$$ 
$$ e_{w2B} = \frac {e_{w2} \cdot \text{bias_axis}}{||\text{bias_axis}||_2^2} *\text{bias_axis}
\tag{8}$$


$$e_{w1B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w1B}} - \mu_B} {|(e_{w1} - \mu_{\perp}) - \mu_B)|} \tag{9}$$


$$e_{w2B}^{corrected} = \sqrt{ |{1 - ||\mu_{\perp} ||^2_2} |} * \frac{e_{\text{w2B}} - \mu_B} {|(e_{w2} - \mu_{\perp}) - \mu_B)|} \tag{10}$$

$$e_1 = e_{w1B}^{corrected} + \mu_{\perp} \tag{11}$$
$$e_2 = e_{w2B}^{corrected} + \mu_{\perp} \tag{12}$$


**(4) 연습 과제** : 아래 함수를 구현합니다. 위의 방정식을 사용하여 단어 쌍의 최종 균등화 버전을 얻으십시오. 행운을 빕니다!

In [13]:
def equalize(pair, bias_axis, word_to_vec_map):
    """
    Debias gender specific words by following the equalize method described in the figure above.
    
    Arguments:
    pair -- pair of strings of gender specific words to debias, e.g. ("actress", "actor") 
    bias_axis -- numpy-array of shape (50,), vector corresponding to the bias axis, e.g. gender
    word_to_vec_map -- dictionary mapping words to their corresponding vectors
    
    Returns
    e_1 -- word vector corresponding to the first word
    e_2 -- word vector corresponding to the second word
    """
    
    # Step 1: Select word vector representation of "word". Use word_to_vec_map. (≈ 2 lines)
    w1, w2 = pair[0],pair[1]
    e_w1, e_w2 = word_to_vec_map[w1],word_to_vec_map[w2]
    
    # Step 2: Compute the mean of e_w1 and e_w2 (≈ 1 line)
    mu = (e_w1 + e_w2)/2

    # Step 3: Compute the projections of mu over the bias axis and the orthogonal axis (≈ 2 lines)
    mu_B = (np.dot(mu,bias_axis)/np.linalg.norm(bias_axis)**2)*bias_axis
    mu_orth = mu-mu_B

    # Step 4: Use equations (7) and (8) to compute e_w1B and e_w2B (≈2 lines)
    e_w1B = (np.dot(e_w1,bias_axis)/np.linalg.norm(bias_axis)**2)*bias_axis
    e_w2B = (np.dot(e_w2,bias_axis)/np.linalg.norm(bias_axis)**2)*bias_axis
        
    # Step 5: Adjust the Bias part of e_w1B and e_w2B using the formulas (9) and (10) given above (≈2 lines)
    corrected_e_w1B = np.sqrt(np.abs(1-np.linalg.norm(mu_orth)**2))*((e_w1B - mu_B)/np.abs((e_w1-mu_orth)-mu_B))
    corrected_e_w2B = np.sqrt(np.abs(1-np.linalg.norm(mu_orth)**2))*((e_w2B - mu_B)/np.abs((e_w2-mu_orth)-mu_B))

    # Step 6: Debias by equalizing e1 and e2 to the sum of their corrected projections (≈2 lines)
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth
    
    return e1, e2

In [14]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.11711095765336832
cosine_similarity(word_to_vec_map["woman"], gender) =  0.35666618846270376

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7165727525843935
cosine_similarity(e2, gender) =  0.7396596474928909


**Expected Output**:

cosine similarities before equalizing:
<table>
    <tr>
        <td>
            cosine_similarity(word_to_vec_map["man"], gender) =
        </td>
        <td>
         -0.117110957653
        </td>
    </tr>
        <tr>
        <td>
            cosine_similarity(word_to_vec_map["woman"], gender) =
        </td>
        <td>
         0.356666188463
        </td>
    </tr>
</table>

cosine similarities after equalizing:
<table>
    <tr>
        <td>
            **cosine_similarity(u1, gender)** =
        </td>
        <td>
         -0.700436428931
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(u2, gender)** =
        </td>
        <td>
         0.700436428931
        </td>
    </tr>
</table>

다른 단어 쌍에 이퀄라이제이션을 적용하려면 위의 셀에있는 입력 단어를 자유롭게 사용하십시오.

이러한 편향성 제거 알고리즘은 편향을 줄이는 데 매우 유용하지만 완벽하지는 않으며 편향의 모든 흔적을 제거하지는 않습니다. 예를 들어,이 구현의 한 가지 약점은 편향 방향 $ g $가 단어 _woman_ 및 _man_의 쌍만 사용하여 정의되었다는 것입니다. 앞에서 설명한 것처럼 $ g$ 가 $g_1 = e_{woman} - e_{man}$; $g_2 = e_{mother} - e_{father}$; $g_3 = e_{girl} - e_{boy}$; 그리고 이들을 평균화하면 50 차원 단어 임베딩 공간에서 "성별"차원에 대한 더 나은 추정치를 얻을 수 있습니다. 이러한 변형도 자유롭게 플레이하십시오.

### 축하합니다

이 노트북의 끝까지 왔으며 단어 벡터를 사용하고 수정할 수 있는 많은 방법을 보았습니다.

이 노트를 완성한 것을 축하합니다!

**References**:
- The debiasing algorithm is from Bolukbasi et al., 2016, [Man is to Computer Programmer as Woman is to
Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings.pdf)
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)
